In [1]:
import argparse
import os
import sys

import numpy as np
import pytorch_lightning as pl
import torch
import torchvision

# sys.path.append("../audiodiffusion/")
# print(sys.path)

import audiodiffusion.mel as mel
from librosa.util import normalize
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.utilities.distributed import rank_zero_only
from torch.utils.data import DataLoader, Dataset
import librosa

from audiodiffusion.utils import convert_ldm_to_hf_vae

2023-05-26 07:41:25.337276: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 07:41:25.365212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 07:41:25.893075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tqdm.auto import tqdm

In [7]:
mel_ = mel.Mel()
input_folder = "../project/data/development/"
output_folder = "../project/data/development-mel/"

os.makedirs(output_folder, exist_ok=True)

file_names = os.listdir(input_folder)

for file_name in tqdm(file_names):
    file_path = os.path.join(input_folder, file_name)

    mel_.load_audio(file_path)
    S = librosa.feature.melspectrogram(
        y=mel_.audio,
        sr=mel_.sr,
        n_fft=mel_.n_fft,
        hop_length=mel_.hop_length,
        n_mels=mel_.n_mels,
    )
    log_S = librosa.power_to_db(S, ref=np.max, top_db=mel_.top_db)
    bytedata = (
        ((log_S + mel_.top_db) * 255 / mel_.top_db).clip(0, 255) + 0.5
    ).astype(np.uint8)

    image = bytedata.reshape((bytedata.shape[0], bytedata.shape[1], 1))
    image = image[: mel_.x_res, : mel_.y_res, :]
    image = (image / 255) * 2 - 1

    output_path = os.path.join(output_folder, file_name + ".npy")
    np.save(output_path, image)

  0%|          | 0/2893 [00:00<?, ?it/s]

In [5]:
mel_spec = np.load("../project/data/development-mel/04_Kirkenes_Round street crossing, quiet steps, car, birds, seagulls 1329.wav.npy")

In [8]:
mel_spec.shape

(256, 256, 1)

In [10]:
mel_folder = "../project/data/development-mel/"
caption_folder = "../project/data/development-latent/"
input_folder = "../project/data/development/"
output_folder = "../project/data/development-full/"
os.makedirs(output_folder, exist_ok=True)

file_names = os.listdir(input_folder)
for i, file_name in enumerate(tqdm(file_names)):
    file_name_cropped = file_name.split(".")[0]

    try:
        mel_spec = np.load(os.path.join(mel_folder, file_name_cropped + ".wav.npy"))
        caption = np.load(os.path.join(caption_folder, file_name_cropped + ".npz"))["text_embed"]
    except FileNotFoundError:
        continue
    
    np.savez(os.path.join(output_folder, file_name_cropped + ".npz"), image=mel_spec, text_embed=caption)

  0%|          | 0/2893 [00:00<?, ?it/s]